# **ERA5 Synoptic Winds for SWEX IOP #10**
## This notebook performs the following tasks:
> - #### Makes plot of 850hPa winds over a portion of California using ERA5 reanalysis data for six different times during SWEX IOP #10.

## **Import packages**
#### Links to documentation for packages
> - #### [pathlib](https://docs.python.org/3/library/pathlib.html) | [pygrib](https://jswhit.github.io/pygrib/) | [numpy](https://numpy.org/doc/1.21/) | [pandas](https://pandas.pydata.org/docs/) | [cartopy](https://scitools.org.uk/cartopy/docs/latest/) | [matplotlib](https://matplotlib.org/3.5.3/index.html) |  
> - #### Documentation for packages linked above should mostly correspond to the most stable versions, which may not be the exact versions used when creating this notebook.
> - #### Comments are also included in the actual code cells. Some comments contain links that point to places where I copied or adapted code to fit my needs. Although I tried to these include links for all instancs of copying, it is possible that there may code snippets that I did not do this for.

In [ ]:
#-----------------------------------------------------
#Entire packages
import pathlib
import pygrib
import numpy as np
import pandas as pd

#cartopy imports
import cartopy.crs as ccrs

#matplotlib imports
import matplotlib.patheffects
import matplotlib.font_manager
import matplotlib.pyplot as plt

#Mapping function notebook
%run ./functions_swex_iop_10.ipynb
#-----------------------------------------------------

## **Define variables that point to paths for relevant data**
### Data Information
> - #### ERA5 Reanalysis: [Hourly Data on Pressure Levels (1959-Present)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview)
>> - #### Variables available in downloaded pressure level files (order matters): Geopotential height, u and v wind components, specific humidity, temperature, and relative humidity at specific pressure level indicated by file name. 
>> - #### Pressure levels plotted: 850hPa
>> - #### Filename convention: era5_pressure_level_PPP_YYMMDDHH_utc; Example of 850hPa file: era5_pressure_level_850_2022051308_utc.grib; PPP=Pressure level; Time zone of filenames is UTC.

In [ ]:
#-----------------------------------------------------
#Define a date list which holds date strings for the files we want to read in
date_list = ['2022051217', '2022051221', '2022051301',
             '2022051305', '2022051309', '2022051313']

#Grab files we want based on glob pattern and list comprehension
#ChatGPT came up with this solution for me.
glob_era5_pressure_level_850_files = [file for date in date_list for file in sorted(pathlib.Path("./SWEX2022_datasets/ERA5/pressure_level_850/").glob(f"*{date}*"))]

#Display first few glob paths to ensure we got the files we want
display(glob_era5_pressure_level_850_files)
#-----------------------------------------------------

## **Plot ERA5 850hPa winds during six different times during SWEX IOP 10**
### Notes
> - #### See in-line comments for additional details.

In [ ]:
#-----------------------------------------------------
#Define a list which 
                                         
#Define cartopy plot domain in lat/lon coordinates
lon_lat_tick_num = [4, 4]
lon_lat_extent   = [-123, -115, 33, 39]
lon_lat_ticks    = [-123, -115, 33, 39]

#Define lon/lat extents for cropping our ERA5 data
#I usually crop data a bit bigger than the plotting domain, just to account for any continuity issues for specific variables
#Note: Because we are over the US west coast, and thus using negative longitudes, we subtract degrees on the left longitude and add degrees to the right longitude to crop our data beyond our map
left_lon_crop  = lon_lat_extent[0]-0.25
right_lon_crop = lon_lat_extent[1]+0.25
lower_lat_crop = lon_lat_extent[2]-0.25
upper_lat_crop = lon_lat_extent[3]+0.25

#Define map and data coordinate reference system for our cartopy map
plot_crs = ccrs.PlateCarree()
data_crs = ccrs.PlateCarree()

#Define font properties for different items
fontdict_title_labels    = {'fontsize': 24, 'fontweight': 'bold',   'fontname': 'Nimbus Roman'}
fontdict_text_color_bar  = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_text_annotation = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_tick_labels     = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_quiver_key      = {'size': 14, 'weight': 'bold', 'family': 'Nimbus Roman'}
fontdict_legend_labels   = {'size': 16, 'weight': 'bold', 'family':'Nimbus Roman'}

#Read in csv file with locations for stations in Santa Barbara County during SWEX
instrument_metadata_file = pd.read_csv(pathlib.Path('./SWEX2022_datasets/geographic_files/isfs_raws_asos_lidar_radio_station_metadata.csv'))

#Index the pandas csv DataFrame to find lat/lons of the stations we want
ksmx_df  = instrument_metadata_file.loc[instrument_metadata_file['name']=='KSMX']
kbfl_df  = instrument_metadata_file.loc[instrument_metadata_file['name']=='KBFL']
ksba_df  = instrument_metadata_file.loc[instrument_metadata_file['name']=='KSBA']

#Define a list which will hold the letters that are used to title each subplot
subplot_letter_list = ['a)', 'b)', 'c)', 'd)', 'e)', 'f)']


#Run cartopy plot helper function
fig, axs = cartopy_basemap_subplots(plot_crs=plot_crs, data_crs=data_crs, fig_size=(15,20), 
                                   nrows=3, ncols=2, wspace_float=None, hspace_float=None,
                                   lon_lat_extent=lon_lat_extent, lon_lat_ticks=lon_lat_ticks, lon_lat_tick_num=[5,4], 
                                   lon_lat_ticks_on=True, xtick_ytick_set_list=[False, False, False, False, True, False],
                                   high_res_coastline=False, 
                                   high_res_wrf_topo_sb_bool=False, 
                                   low_res_wrf_topo_sb_bool=False, 
                                   low_res_wrf_topo_ca_bool=True, 
                                   wrf_topo_colorbar_each_plot_bool=False, 
                                   wrf_topo_colorbar_entire_figure_bool=True,
                                   scale_bar_bool=False, scale_bar_position=None, scale_bar_length=None, 
                                   inset_ca_bool=False, inset_bbox_position=None, 
                                   ocean_color='lightskyblue')
#-----------------------------------------------------
#For every file in the list of daily files do the following:
for index, (ax, era5_pressure_level_850_file) in enumerate(zip(axs.flatten(), glob_era5_pressure_level_850_files)):

    #Open grib file
    grbs_pressure_level_850 = pygrib.open(str(era5_pressure_level_850_file))

    #Select each parameter for analysis & plotting (pressure level file)
    grb_pressure_level_850_u_wind = grbs_pressure_level_850.select()[1] #u wind at pressure level (unit = m/s)
    grb_pressure_level_850_v_wind = grbs_pressure_level_850.select()[2] #v wind at pressure level (unit = m/s)

    #Grab lat/lon/values for each pressure level parameter (area was cropped during download of data)
    data_pressure_level_850_u_wind, lats_pressure_level_850_u_wind, lons_pressure_level_850_u_wind = grb_pressure_level_850_u_wind.data(lat1=lower_lat_crop, lat2=upper_lat_crop, lon1=left_lon_crop, lon2=right_lon_crop)
    data_pressure_level_850_v_wind, lats_pressure_level_850_v_wind, lons_pressure_level_850_v_wind = grb_pressure_level_850_v_wind.data(lat1=lower_lat_crop, lat2=upper_lat_crop, lon1=left_lon_crop, lon2=right_lon_crop)

    #Plot wind barbs for every n-th grid point
    #Note that ERA5 winds have a default unit of m/s. We convert from m/s to knots below by multiplying the entire u and v component arrays by 1.94383.
    #https://earthscience.stackexchange.com/questions/19001/plotting-era5-u-v-wind-data-using-python
    nth_pt = 1

    #Plot vectors for 850hPa winds
    quiver = ax.quiver(lons_pressure_level_850_u_wind[::nth_pt,::nth_pt], 
                       lats_pressure_level_850_u_wind[::nth_pt,::nth_pt], 
                       data_pressure_level_850_u_wind[::nth_pt,::nth_pt], 
                       data_pressure_level_850_v_wind[::nth_pt,::nth_pt], 
                       angles='xy', scale_units='xy', scale=20,
                       pivot='middle',  zorder=2, transform=data_crs)

    #Add title with time/date information
    datetime_utc     = pd.to_datetime(f'{str(era5_pressure_level_850_file)[-19:-15]}-{str(era5_pressure_level_850_file)[-15:-13]}-{str(era5_pressure_level_850_file)[-13:-11]} {str(era5_pressure_level_850_file)[-11:-9]}')
    datetime_pdt     = datetime_utc-pd.Timedelta(7, unit='h')
    datetime_pdt_str = f'{str(datetime_pdt.hour).zfill(2)}00 PDT {str(datetime_pdt.day).zfill(2)} May 2022'
    ax.set_title(f'{subplot_letter_list[index]} {datetime_pdt_str}', **fontdict_title_labels)
    
    #Add points for KSMX, KBFL, and KSBA
    ax.scatter(ksmx_df['longitude'], ksmx_df['latitude'], s=150,  marker='o', color='blue',   edgecolor='black', label='KSMX', transform=data_crs, zorder=10)
    ax.scatter(kbfl_df['longitude'], kbfl_df['latitude'], s=150,  marker='o', color='red',    edgecolor='black', label='KBFL', transform=data_crs, zorder=10)
    ax.scatter(ksba_df['longitude'], ksba_df['latitude'], s=150,  marker='o', color='black',  edgecolor='black', label='KSBA', transform=data_crs, zorder=10)

    #Add stuff to specific subplot
    if index == 4:
        
        #Add legend
        ax.legend(loc='center', bbox_to_anchor=(0.87, 0.78),
                  ncols=1, markerscale=1, handletextpad=0.2, 
                  fancybox=True, shadow=True, framealpha=1, 
                  facecolor='snow', edgecolor='black', prop=fontdict_legend_labels)
        
        #Define inset axis for quiver key
        ax1 = inset_axes(ax, width='100%',height='100%',loc='upper right', borderpad=0, bbox_to_anchor=(0.75, 0.94, 0.25, 0.06), bbox_transform=ax.transAxes)

        #Remove ticks from inset axis
        ax1.tick_params(labelleft=False,labelbottom=False,left=False,bottom=False)

        #Add quiver key to inset axis
        ax1.quiverkey(quiver, X=0.825, Y=0.965, U=10, label=r'10 m s$^{-1}$', labelpos='E', fontproperties=fontdict_quiver_key)
#-----------------------------------------------------
#Save figure    
plt.savefig(f'./figures/figure_03_era5_ca_winds.png', bbox_inches='tight', dpi=500)

#Show figure
plt.show()
#-----------------------------------------------------